<a href="https://colab.research.google.com/github/khoadangtruong/Pytorch/blob/master/Inception_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import glob
import pathlib
import torch
import numpy as np
import pandas as pd
from io import open
from PIL import Image
from torch import nn, optim, from_numpy, flatten
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets

In [ ]:
print(torch.__version__, torch.cuda.is_available())

1.9.0+cu102 True


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/scene.zip' -d '/content/data'

In [ ]:
train_path = '/content/data/scene/seg_train/seg_train'
test_path = '/content/data/scene/seg_test/seg_test'
pred_path = '/content/data/scene/seg_pred/seg_pred'

In [ ]:
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
print(classes, len(classes))

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street'] 6


In [ ]:
transformers = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5]
    )
])

In [ ]:
train_loader = DataLoader(
    dataset = datasets.ImageFolder(root = train_path, transform = transformers),
    batch_size = 128,
    shuffle = True
)

test_loader = DataLoader(
    dataset = datasets.ImageFolder(root = test_path, transform = transformers),
    batch_size = 128,
    shuffle = True
)

In [ ]:
import torch.nn.functional as F

In [ ]:
class InceptionA(nn.Module):

    def __init__(self, in_channels):
        super(InceptionA, self).__init__()

        self.branch1x1 = nn.Conv2d(
            in_channels = in_channels, out_channels = 16, 
            kernel_size = 1, stride = 1, padding = 1
        )

        self.branch5x5_1 = nn.Conv2d(
            in_channels = in_channels, out_channels = 16,
            kernel_size = 1, stride = 1, padding = 1
        )
        self.branch5x5_2 = nn.Conv2d(
            in_channels = 16, out_channels = 24,
            kernel_size = 5, stride = 1, padding = 2
        )

        self.branch3x3_1 = nn.Conv2d(
            in_channels = in_channels, out_channels = 16,
            kernel_size = 1, stride = 1, padding = 1
        )
        self.branch3x3_2 = nn.Conv2d(
            in_channels = 16, out_channels = 24,
            kernel_size = 3, stride = 1, padding = 1
        )
        self.branch3x3_3 = nn.Conv2d(
            in_channels = 24, out_channels = 32,
            kernel_size = 3, stride = 1, padding = 1
        )

        self.branch_pool = nn.Conv2d(
            in_channels = in_channels, out_channels = 24,
            kernel_size = 1, stride = 1, padding = 1
        )
    
    def forward(self, x):
        
        branch1x1 = self.branch1x1(x)

        branch5x5_1 = self.branch5x5_1(x)
        branch5x5_2 = self.branch5x5_2(branch5x5_1)

        branch3x3_1 = self.branch3x3_1(x)
        branch3x3_2 = self.branch3x3_2(branch3x3_1)
        branch3x3_3 = self.branch3x3_3(branch3x3_2)

        avg_pool = F.avg_pool2d(input = x, kernel_size = 3, stride = 1, padding = 1)
        branch_pool = self.branch_pool(avg_pool)

        outputs = [branch1x1, branch5x5_2, branch3x3_3, branch_pool]
        return torch.cat(outputs, 1)

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()

        self.conv1 = nn.Conv2d(
            in_channels = 3, out_channels = 16,
            kernel_size = 5, stride = 1, padding = 1
        )
        self.bn1 = nn.BatchNorm2d(num_features = 16, momentum = 0.5)

        self.conv2 = nn.Conv2d(
            in_channels = 96, out_channels = 24,
            kernel_size = 5, stride = 1, padding = 1
        )
        self.bn2 = nn.BatchNorm2d(num_features = 24, momentum = 0.5)

        self.conv3 = nn.Conv2d(
            in_channels = 96, out_channels = 32,
            kernel_size = 5, stride = 1, padding = 1
        )
        self.bn3 = nn.BatchNorm2d(num_features = 32, momentum = 0.5)

        self.inception1 = InceptionA(16)
        self.inception2 = InceptionA(24)
        self.inception3 = InceptionA(32)

        self.max_pool = nn.MaxPool2d(
            kernel_size = 2, stride = 2 
        )

        self.fc1 = nn.Linear(38400, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 6)

        self.relu = nn.ReLU()

    def forward(self, x):

        x = self.relu(self.max_pool(self.bn1(self.conv1(x))))
        x = self.inception1(x)
        x = self.relu(self.max_pool(self.bn2(self.conv2(x))))
        x = self.inception2(x)
        x = self.relu(self.max_pool(self.bn3(self.conv3(x))))
        x = self.inception3(x)
        
        x = flatten(x, 1)

        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.fc5(x)

        return x



In [ ]:
model = Model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = model.parameters(), lr = 1e-3, weight_decay = 1e-4)

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = Variable(data.cuda()), Variable(target.cuda())
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print(f'Epoch: {epoch + 1} | Batch: {batch_idx * len(data)}/{len(train_loader.dataset)} | Loss: {loss.item():.4f}')

In [ ]:
def test():
    model.eval()
    l_sum = 0
    correct = 0
    best_accuracy = 0.0
    with torch.no_grad():
        for data, target in test_loader:
            if torch.cuda.is_available():
                data, target = Variable(data.cuda()), Variable(target.cuda())
            outputs = model(data)
            loss = criterion(outputs, target)
            l_sum += loss.item()
            pred = outputs.data.max(1, keepdim = True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    avg_loss = l_sum / len(test_loader.dataset)
    test_accuracy = 100. * (correct / len(test_loader.dataset))
    print(f'Average loss: {avg_loss:.4f} | Accuracy: {test_accuracy:.0f}%')

    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), '/content/checkpoint.model')
        best_accuracy = test_accuracy

In [ ]:
for epoch in range(20):
    train(epoch)
    test()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 1 | Batch: 0/14034 | Loss: 1.7808
Epoch: 1 | Batch: 1280/14034 | Loss: 1.3918
Epoch: 1 | Batch: 2560/14034 | Loss: 1.0904
Epoch: 1 | Batch: 3840/14034 | Loss: 1.0234
Epoch: 1 | Batch: 5120/14034 | Loss: 0.8462
Epoch: 1 | Batch: 6400/14034 | Loss: 0.8332
Epoch: 1 | Batch: 7680/14034 | Loss: 0.7678
Epoch: 1 | Batch: 8960/14034 | Loss: 0.7223
Epoch: 1 | Batch: 10240/14034 | Loss: 0.6542
Epoch: 1 | Batch: 11520/14034 | Loss: 0.6625
Epoch: 1 | Batch: 12800/14034 | Loss: 0.8422
Average loss: 0.0057 | Accuracy: 73%
Epoch: 2 | Batch: 0/14034 | Loss: 0.5832
Epoch: 2 | Batch: 1280/14034 | Loss: 0.8443
Epoch: 2 | Batch: 2560/14034 | Loss: 0.7352
Epoch: 2 | Batch: 3840/14034 | Loss: 0.6953
Epoch: 2 | Batch: 5120/14034 | Loss: 0.7467
Epoch: 2 | Batch: 6400/14034 | Loss: 0.5684
Epoch: 2 | Batch: 7680/14034 | Loss: 0.6558
Epoch: 2 | Batch: 8960/14034 | Loss: 0.5191
Epoch: 2 | Batch: 10240/14034 | Loss: 0.7007
Epoch: 2 | Batch: 11520/14034 | Loss: 0.5838
Epoch: 2 | Batch: 12800/14034 | Loss: 0.

In [ ]:
checkpoint = torch.load('/content/checkpoint.model')
model = Model()
model.load_state_dict(checkpoint)
model.eval()

Model(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.5, affine=True, track_running_stats=True)
  (conv2): Conv2d(96, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(24, eps=1e-05, momentum=0.5, affine=True, track_running_stats=True)
  (conv3): Conv2d(96, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.5, affine=True, track_running_stats=True)
  (inception1): InceptionA(
    (branch1x1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    (branch5x5_1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    (branch5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (branch3x3_1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    (branch3x3_2): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (branch3x3_3): Conv2d(24, 32, kernel_size

In [ ]:
def prediction(img_path,transformer):

    image=Image.open(img_path)
    image_tensor = transformer(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    index=output.data.numpy().argmax()
    pred=classes[index]
    
    return pred

In [ ]:
images_path = glob.glob(pred_path + '/*.jpg')

In [ ]:
pred_dict = {}
for i in images_path:
    pred_dict[i[i.rfind('/')+1:]] = prediction(i,transformers)

In [ ]:
pred_dict

{'11361.jpg': 'glacier',
 '23134.jpg': 'glacier',
 '5493.jpg': 'mountain',
 '20193.jpg': 'buildings',
 '11589.jpg': 'forest',
 '20097.jpg': 'mountain',
 '9326.jpg': 'mountain',
 '1307.jpg': 'sea',
 '19442.jpg': 'forest',
 '22751.jpg': 'street',
 '2055.jpg': 'glacier',
 '3258.jpg': 'sea',
 '22697.jpg': 'mountain',
 '212.jpg': 'street',
 '6806.jpg': 'mountain',
 '17599.jpg': 'glacier',
 '20526.jpg': 'mountain',
 '18069.jpg': 'buildings',
 '12587.jpg': 'buildings',
 '13414.jpg': 'forest',
 '18404.jpg': 'buildings',
 '6727.jpg': 'mountain',
 '14137.jpg': 'street',
 '14855.jpg': 'glacier',
 '7883.jpg': 'glacier',
 '8174.jpg': 'sea',
 '10914.jpg': 'buildings',
 '17357.jpg': 'mountain',
 '4525.jpg': 'glacier',
 '198.jpg': 'mountain',
 '10054.jpg': 'glacier',
 '22052.jpg': 'forest',
 '11463.jpg': 'sea',
 '23341.jpg': 'mountain',
 '4272.jpg': 'street',
 '19788.jpg': 'glacier',
 '2938.jpg': 'glacier',
 '12271.jpg': 'buildings',
 '22714.jpg': 'forest',
 '7957.jpg': 'buildings',
 '18836.jpg': 'for